In [1]:
# using vgg16 for face recognition

In [2]:
from keras.applications import VGG16
# MobileNet was designed to work on 224 x 224 pixel input images sizes
img_rows, img_cols = 224, 224 

# Re-loads the MobileNet model without the top or FC layers
VGG16 = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in VGG16.layers:
    layer.trainable = False

Using TensorFlow backend.


In [3]:
def add(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""
    top_model = VGG16.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

# Set our class number to 3 (Young, Middle, Old)
num_classes = 2

FC_Head = add(VGG16, num_classes)

model = Model(inputs = VGG16.input, outputs = FC_Head)

print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [5]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'C:/Users/Tushar/Desktop/mlops/Face/Train'
validation_data_dir = 'C:/Users/Tushar/Desktop/mlops/Face/Test'

# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# set our batch size (typically on most mid tier systems we'll use 16-32)
batch_size = 32
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')

Found 189 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [6]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

                     
checkpoint = ModelCheckpoint("face_recognition.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 189
nb_validation_samples = 100

# We only train 5 EPOCHS 
epochs = 5
batch_size = 16

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

Epoch 1/5
11/11 [==============================] - 149s 14s/step - loss: 1.0991 - accuracy: 0.5260 - val_loss: 0.4536 - val_accuracy: 0.5366

Epoch 00001: val_loss improved from inf to 0.45358, saving model to face_recognition.h5
Epoch 2/5
11/11 [==============================] - 140s 13s/step - loss: 0.2305 - accuracy: 0.9162 - val_loss: 0.1464 - val_accuracy: 0.9412

Epoch 00002: val_loss improved from 0.45358 to 0.14641, saving model to face_recognition.h5
Epoch 3/5
11/11 [==============================] - 147s 13s/step - loss: 0.4008 - accuracy: 0.9220 - val_loss: 0.1022 - val_accuracy: 1.0000

Epoch 00003: val_loss improved from 0.14641 to 0.10223, saving model to face_recognition.h5
Epoch 4/5
11/11 [==============================] - 139s 13s/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 1.9016 - val_accuracy: 0.4926

Epoch 00004: val_loss did not improve from 0.10223
Epoch 5/5
11/11 [==============================] - 128s 12s/step - loss: 0.3395 - accuracy: 0.9054 - val_loss

In [7]:
from keras.models import load_model

classifier = load_model('face_recognition.h5')

In [9]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

face_reco = {"[0]": "Nikky ", 
                      "[1]": "Tushar",
                      }
face_reco_n = {"Nikky": "NIKITA ", 
                      "Tushar": "TUSHAR",
                      }

def draw_test(name, pred, im):
    face = face_reco[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, face, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + face_reco_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,10):
    input_im = getRandomImage("C:/Users/Tushar/Desktop/mlops/Face/Test/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)

cv2.destroyAllWindows()

Class - NIKITA 
Class - TUSHAR
Class - NIKITA 
Class - NIKITA 
Class - NIKITA 
Class - TUSHAR
Class - TUSHAR
Class - NIKITA 
Class - NIKITA 
Class - NIKITA 
